In [1]:
from datetime import datetime
from heapq import nlargest
from operator import itemgetter
from collections import defaultdict
from sys import stdout
print('Preparing arrays...')

Preparing arrays...


In [2]:
###### finding most popular 50
f = open("train.csv", "r")
f.readline()
most_hotels_cntr_hm_sdi = defaultdict(int)
total = 0

In [3]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '':
        most_hotels_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)] += 1

f.close()
end = time.time()
print(end - start)

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
121.888365984


In [4]:
topitems = nlargest(50, most_hotels_cntr_hm_sdi.items(), key=itemgetter(1))

In [5]:
topitems_dict = defaultdict(int)

In [6]:
for k,v in topitems:
    topitems_dict[k] = v

In [11]:
############ Features transformation & Construction #############
f = open("train.csv", "r")
f.readline()
feature1_cntr_hm_sdi = defaultdict(lambda: defaultdict(list)) # feature_contstrunction
total = 0
missline = 0

In [12]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '' and ((hotel_country,hotel_market,srch_destination_id) in topitems_dict):
        try:
            search_time_DT = datetime.strptime(arr[0], '%Y-%m-%d %H:%M:%S')
            arr[0] = search_time_DT.hour      #srch_time_hour
            arr[1] = search_time_DT.weekday() #srch_time_weekday
            arr[20] = search_time_DT.year     #srch_time_year
            #arr[2] = int(arr[2])             #posa_continent
            
            arr[6] = float(arr[6]) #orig_dest_distance
        
            #arr[8] = int(arr[8]) #is_mobile
            #arr[9] = int(arr[9]) #is_package
            #arr[10] = int(arr[10]) #channel
        
            checkInDateDT = datetime.strptime(arr[11], "%Y-%m-%d")
            checkOutDateDT = datetime.strptime(arr[12], "%Y-%m-%d") 
            arr[11] = checkInDateDT.timetuple().tm_yday + 1  #srch_ci_day
            arr[12] = checkOutDateDT.timetuple().tm_yday+ 1 #srch_co_day

            arr[13] = int(arr[13]) #srch_adult_count
            arr[14] = int(arr[14]) #srch_child_count
            arr[15] = int(arr[15]) #srch_room_count
            
            arr[16] = arr[12] - arr[11] #duration
            #arr[17] = int(arr[17]) #srch_dest_type_id
            arr[18] = int(arr[18])  #is_booking should be weighted
            arr[19] = int(arr[19])  #hotel_cluster
        except:
            missline += 1
            continue
        colNames = ['hour','dow','posa','','','','dist','','mobile','package','channel','ci','co',
                    'adults','children','room','stay','type','booking','cluster','year']
        colindex = [0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20]
        for i in colindex:
            feature1_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)][colNames[i]].append(arr[i])

f.close()
end = time.time()
print(end - start)

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
653.239267111


In [20]:
import collections as cl

In [21]:
import time
start = time.time()

length = len(feature1_cntr_hm_sdi.items())
for i in range(length):
    temp_ci = feature1_cntr_hm_sdi.items()[i][1]['ci']
    temp_co = feature1_cntr_hm_sdi.items()[i][1]['co']
    counter_ci = cl.Counter(temp_ci)
    counter_co = cl.Counter(temp_co)
    feature1_cntr_hm_sdi.items()[i][1]['ci'] = [counter_ci[k] for k in temp_ci]
    feature1_cntr_hm_sdi.items()[i][1]['co'] = [counter_co[k] for k in temp_co]
end = time.time()
print(end - start)

11.8695738316


In [18]:
feature1_cntr_hm_sdi.items()[0][1]['ci']

[224,
 306,
 188,
 199,
 199,
 67,
 67,
 149,
 301,
 301,
 301,
 242,
 242,
 242,
 279,
 279,
 279,
 279,
 279,
 65,
 65,
 65,
 65,
 65,
 65,
 65,
 65,
 69,
 284,
 284,
 284,
 360,
 360,
 193,
 193,
 204,
 204,
 236,
 365,
 203,
 203,
 203,
 203,
 203,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 224,
 181,
 181,
 263,
 263,
 279,
 279,
 93,
 93,
 93,
 93,
 93,
 93,
 235,
 235,
 235,
 235,
 220,
 229,
 229,
 234,
 229,
 229,
 229,
 242,
 209,
 209,
 222,
 222,
 45,
 298,
 325,
 348,
 348,
 348,
 348,
 348,
 348,
 348,
 165,
 165,
 165,
 305,
 305,
 305,
 319,
 319,
 250,
 250,
 250,
 250,
 250,
 250,
 117,
 117,
 189,
 189,
 189,
 189,
 189,
 189,
 263,
 263,
 263,
 263,
 263,
 262,
 256,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 263,
 256,
 262,
 263,
 263,
 263,
 263,
 263,
 263,
 262,
 263,
 263,
 263,
 263,
 263,
 263,
 263,
 263,
 263,
 257,
 257,
 257,
 257,
 39,
 130,
 130,
 210,
 210,
 210,
 210,
 210,
 210,
 210,
 210,
 210,
 210,

In [22]:
feature1_cntr_hm_sdi.items()[0][1]['ci']

[458,
 310,
 286,
 408,
 408,
 180,
 180,
 106,
 112,
 112,
 112,
 778,
 778,
 778,
 204,
 204,
 204,
 204,
 204,
 135,
 135,
 135,
 135,
 135,
 135,
 135,
 135,
 144,
 499,
 499,
 499,
 675,
 675,
 454,
 454,
 356,
 356,
 619,
 407,
 353,
 353,
 353,
 353,
 353,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 458,
 415,
 415,
 469,
 469,
 204,
 204,
 190,
 190,
 190,
 190,
 190,
 190,
 616,
 616,
 616,
 616,
 601,
 700,
 700,
 531,
 700,
 700,
 700,
 778,
 405,
 405,
 742,
 742,
 244,
 436,
 173,
 265,
 265,
 265,
 265,
 265,
 265,
 265,
 315,
 315,
 315,
 216,
 216,
 216,
 237,
 237,
 401,
 401,
 401,
 401,
 401,
 401,
 189,
 189,
 362,
 362,
 362,
 362,
 362,
 362,
 469,
 469,
 469,
 469,
 469,
 361,
 370,
 361,
 361,
 361,
 361,
 361,
 361,
 361,
 469,
 370,
 361,
 469,
 469,
 469,
 469,
 469,
 469,
 361,
 469,
 469,
 469,
 469,
 469,
 469,
 469,
 469,
 469,
 421,
 421,
 421,
 421,
 104,
 210,
 210,
 506,
 506,
 506,
 506,
 506,
 506,
 506